In [33]:
from torchmetrics.text.rouge import ROUGEScore
rouge = ROUGEScore()
from pprint import pprint

In [34]:
import spacy

In [35]:

from spacy.lang.en.stop_words import STOP_WORDS
import pandas as pd

In [36]:
from summarization_algorithm import * 

In [37]:
df = pd.read_csv("dataset.csv")
df=df.iloc[:,1:]#remove first column(unnamed col)
df.dropna(inplace=True)
columns_titles = ["original","summary"]
df=df.reindex(columns=columns_titles)
mini_df = df[:100]
mini_df

,original,summary
0,The Daman and Diu administration on Wednesday ...,The Administration of Union Territory Daman an...
1,"From her special numbers to TV?appearances, Bo...",Malaika Arora slammed an Instagram user who tr...
2,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Hotels in Mumbai and other Indian cities are t...,Hotels in Maharashtra will train their staff t...
4,An alleged suspect in a kidnapping case was fo...,A 32-year-old man on Wednesday was found hangi...
...,...,...
95,Facing pressure from President Donald Trump to...,The US Air Force is negotiating with Boeing to...
96,"Allahabad, Aug 01 (PTI) The Allahabad High Cou...",The Allahabad High Court on Tuesday said it wo...
97,Motorman Mahendra Prasad called up the railway...,As many as 13 people died while travelling in ...
98,Bored in confines of a sprawling resort in Ben...,The Gujarat Congress MLAs staying in a Bengalu...


In [38]:
# def preprocessing(text):
#     sentences = text

#     # Load the model (English) into spaCy
#     nlp = spacy.load('en_core_web_sm')

#     # Adding 'sentencizer' component to the pipeline
#     nlp.add_pipe('sentencizer')

#     # Tokenization & Lemmatization
#     lemmatized_sentences = []

#     doc = nlp(sentences)

#     sentences = []
#     for sentence in doc.sents:
#         sentences.append(sentence.text)
#         lemmatized_sentences.append([token.lemma_ for token in sentence])


#     # Removing Stop Words & Punctuation 
#     filtered_sentences = []

#     for sentences_group in lemmatized_sentences:
#         filtered = ""

#         for sentence in sentences_group:
#             sentence_doc = nlp(sentence)
#             words_of_sentence = [token.text for token in sentence_doc]

#             for token in sentence_doc:
#                 if token.is_stop == False and token.text.isalpha() == True:
#                     filtered += token.text + " "

#         filtered_sentences.append(filtered)

#     return sentences, filtered_sentences

def preprocessing(text, lemmatization = False):
    sentences = text

    # Load the model (English) into spaCy
    nlp = spacy.load('en_core_web_sm')

    # Adding 'sentencizer' component to the pipeline
    nlp.add_pipe('sentencizer')

    # Tokenization & Lemmatization
    lemmatized_sentences = []

    doc = nlp(sentences)

    sentences = []
    if(lemmatization):
        for sentence in doc.sents:
            sentences.append(sentence.text)
            lemmatized_sentences.append([token.lemma_ for token in sentence])
    else:
        for sentence in doc.sents:
            sentences.append(sentence.text)
            lemmatized_sentences.append([token.text for token in sentence])



    # Removing Stop Words & Punctuation 
    filtered_sentences = []

    for sentences_group in lemmatized_sentences:
        filtered = ""

        for sentence in sentences_group:
            sentence_doc = nlp(sentence)
            words_of_sentence = [token.text for token in sentence_doc]

            for token in sentence_doc:
                if token.is_stop == False and token.text.isalpha() == True:
                    filtered += token.text + " "

        filtered_sentences.append(filtered)

    return sentences, filtered_sentences


In [39]:
rows,columns = mini_df.shape
articles_sentences,filtered_articles =[],[] 
for row in range(100):
    articles_sentence , filtered_article = preprocessing(mini_df.iloc[row,0])
    articles_sentences.append(articles_sentence)
    filtered_articles.append(filtered_article)



In [40]:
def summarize_with(articles_sentences,filtered_articles ,summary_algorithm,size = 2):
    rows = len(articles_sentences)
    rows = 100#uncomment this for runtime speed
    summarized_text = []
    for row in range(rows):
        sentences=articles_sentences[row]
        filtered_sentences = filtered_articles[row]
                                  #(filtered_sentences,sentence)
        summary = summary_algorithm(filtered_sentences,sentences,size)
        summarized_text.append(summary)
    summary_df = pd.DataFrame (summarized_text, columns = [f'{summary_algorithm.__name__} summary'])
    return summary_df

In [41]:
luhn = summarize_with(articles_sentences,filtered_articles ,luhn_algorithm).reset_index(drop=True)


In [42]:
text_matching = summarize_with(articles_sentences,filtered_articles ,text_matching_algorithm).reset_index(drop=True)


In [43]:
lsa = summarize_with(articles_sentences,filtered_articles ,lsa_summarization).reset_index(drop=True)


In [44]:
LexRank = summarize_with(articles_sentences,filtered_articles ,LexRank_algorithm).reset_index(drop=True)

f:\PycharmProjects\NLP\text_summarization\summarization_algorithm.py:195: RuntimeWarning: invalid value encountered in scalar divide
  new_tf_idf[row,col] = tf_idf[row,col]/sent_length[row]


In [45]:
horizontal_concat_df = pd.concat(
    [
        mini_df,
        luhn,
        lsa,
        text_matching,
        LexRank
    ],
    axis=1,
)
#horizontal_concat_df[horizontal_concat_df.isnull().any(axis=1)]
horizontal_concat_df

,original,summary,luhn_algorithm summary,lsa_summarization summary,text_matching_algorithm summary,LexRank_algorithm summary
0,The Daman and Diu administration on Wednesday ...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...,The RSS is the ideological parent of the rulin...,The Daman and Diu administration on Wednesday ...,The union territory?s administration was force...
1,"From her special numbers to TV?appearances, Bo...",Malaika Arora slammed an Instagram user who tr...,A post shared by Malaika Arora Khan (@malaikaa...,(read alimony) money to wear ?short clothes an...,A post shared by Malaika Arora Khan (@malaikaa...,The details of the alimony are only known to M...
2,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...,"In its response, the management of the autonom...",on the marital declaration form be immediately...,IGIMS medical superintendent Dr Manish Mandal ...,on the marital declaration form be immediately...
3,Hotels in Mumbai and other Indian cities are t...,Hotels in Maharashtra will train their staff t...,Human trafficking is the world's fastest growi...,RESCUE ME APPThe Rescue Me app - to be launche...,"Last year, major hotel groups, including the H...",Awsarmmel said hotels would be told about 50 s...
4,An alleged suspect in a kidnapping case was fo...,A 32-year-old man on Wednesday was found hangi...,"A native of Kasganj in UP, Kumar was unmarried...","Kumar was one of them,?Their relationship ende...",A team was sent to Kumar?s village but when he...,said a police officer.said the officer.
...,...,...,...,...,...,...
95,Facing pressure from President Donald Trump to...,The US Air Force is negotiating with Boeing to...,Facing pressure from President Donald Trump to...,Boeing built two 747-8s out of an order of fou...,Facing pressure from President Donald Trump to...,Boeing built two 747-8s out of an order of fou...
96,"Allahabad, Aug 01 (PTI) The Allahabad High Cou...",The Allahabad High Court on Tuesday said it wo...,The Allahabad High Court today said it would h...,"Allahabad, Aug 01 (PTI)The Allahabad High Cour...",The Allahabad High Court today said it would h...,The Allahabad High Court today said it would h...
97,Motorman Mahendra Prasad called up the railway...,As many as 13 people died while travelling in ...,Motorman Mahendra Prasad called up the railway...,"Last year, out of 3,202 deaths recorded, a maj...","According to the railway data, 348 people died...","Crossing tracks led to 1,798 of 3,202 train fa..."
98,Bored in confines of a sprawling resort in Ben...,The Gujarat Congress MLAs staying in a Bengalu...,Following strict orders of the Karnataka Power...,Bored in confines of a sprawling resort in Ben...,Following strict orders of the Karnataka Power...,The MLAs who were allowed to go out are suppos...


In [46]:
def calculate_efficiency(predicted_summary,original_summary):
    rouge = ROUGEScore()
    return rouge(predicted_summary, original_summary)

In [47]:
def df_sentences_efficiency(df,algorithm_summary_df):
    sentences_efficiency = []
    rows, column = algorithm_summary_df.shape
    rows = 100
    for row in range(rows):
        predicted_summary = algorithm_summary_df.iloc[row,0]
        original_summary = df.iloc[row,0]
        efficiency_dict = calculate_efficiency(predicted_summary,original_summary)
        sentences_efficiency.append(efficiency_dict)
              
    dataframe = pd.DataFrame(sentences_efficiency)
    return dataframe

In [48]:
luhn_with_scores = pd.concat([luhn,df_sentences_efficiency(mini_df,luhn)],axis = 1)
luhn_with_scores

,luhn_algorithm summary,rouge1_fmeasure,rouge1_precision,rouge1_recall,rouge2_fmeasure,rouge2_precision,rouge2_recall,rougeL_fmeasure,rougeL_precision,rougeL_recall,rougeLsum_fmeasure,rougeLsum_precision,rougeLsum_recall
0,The Daman and Diu administration on Wednesday ...,tensor(0.3370),tensor(1.),tensor(0.2027),tensor(0.3341),tensor(1.),tensor(0.2005),tensor(0.3370),tensor(1.),tensor(0.2027),tensor(0.3370),tensor(1.),tensor(0.2027)
1,A post shared by Malaika Arora Khan (@malaikaa...,tensor(0.3783),tensor(1.),tensor(0.2333),tensor(0.3758),tensor(1.),tensor(0.2313),tensor(0.3783),tensor(1.),tensor(0.2333),tensor(0.3783),tensor(1.),tensor(0.2333)
2,"In its response, the management of the autonom...",tensor(0.3081),tensor(1.),tensor(0.1821),tensor(0.2998),tensor(0.9839),tensor(0.1768),tensor(0.2152),tensor(0.6984),tensor(0.1272),tensor(0.3081),tensor(1.),tensor(0.1821)
3,Human trafficking is the world's fastest growi...,tensor(0.2613),tensor(1.),tensor(0.1503),tensor(0.2557),tensor(0.9875),tensor(0.1468),tensor(0.1645),tensor(0.6296),tensor(0.0946),tensor(0.2613),tensor(1.),tensor(0.1503)
4,"A native of Kasganj in UP, Kumar was unmarried...",tensor(0.3333),tensor(1.),tensor(0.2000),tensor(0.3263),tensor(0.9872),tensor(0.1954),tensor(0.2532),tensor(0.7595),tensor(0.1519),tensor(0.3333),tensor(1.),tensor(0.2000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Facing pressure from President Donald Trump to...,tensor(0.4046),tensor(1.),tensor(0.2536),tensor(0.3985),tensor(0.9904),tensor(0.2494),tensor(0.4046),tensor(1.),tensor(0.2536),tensor(0.4046),tensor(1.),tensor(0.2536)
96,The Allahabad High Court today said it would h...,tensor(0.8058),tensor(1.),tensor(0.6748),tensor(0.8039),tensor(1.),tensor(0.6721),tensor(0.8058),tensor(1.),tensor(0.6748),tensor(0.8058),tensor(1.),tensor(0.6748)
97,Motorman Mahendra Prasad called up the railway...,tensor(0.3636),tensor(1.),tensor(0.2222),tensor(0.3553),tensor(0.9859),tensor(0.2167),tensor(0.3636),tensor(1.),tensor(0.2222),tensor(0.3636),tensor(1.),tensor(0.2222)
98,Following strict orders of the Karnataka Power...,tensor(0.5430),tensor(1.),tensor(0.3727),tensor(0.5235),tensor(0.9750),tensor(0.3578),tensor(0.3709),tensor(0.6829),tensor(0.2545),tensor(0.5430),tensor(1.),tensor(0.3727)


In [49]:
luhn_with_scores = pd.concat([luhn,df_sentences_efficiency(mini_df,luhn)],axis = 1)
text_matching_with_scores = pd.concat([text_matching,df_sentences_efficiency(mini_df,text_matching)],axis = 1)
lsa_with_scores = pd.concat([lsa,df_sentences_efficiency(mini_df,lsa)],axis = 1)
LexRank_with_scores = pd.concat([LexRank,df_sentences_efficiency(mini_df,LexRank)],axis = 1)


KeyboardInterrupt: 